In [ ]:
from datasets import load_dataset
import json

# Load a dataset (replace with your custom dataset)
dataset = load_dataset("openai/math")

def format_data(example):
    """
    Formats dataset into (problem, CoT solution) pairs.
    """
    return {
        "problem": example["question"],
        "solution": example["solution"]
    }

# Process dataset
formatted_dataset = dataset.map(format_data)
formatted_dataset.save_to_disk("formatted_math_dataset")


In [2]:
import openai

def generate_tir_solution(problem):
    """
    Uses GPT-4 to generate a Tool-Integrated Reasoning (TIR) solution.
    """
    client = openai.OpenAI()

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "Generate a step-by-step solution for the math problem, with Python code for execution."},
            {"role": "user", "content": problem}
        ],
        temperature=0.7
    )
    return response["choices"][0]["message"]["content"]

# Example usage
problem = "Find the determinant of the matrix [[2,3],[4,5]]."
solution = generate_tir_solution(problem)
print(solution)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable